In [1]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-

In [2]:
import logging, time, sys, decimal, OpenOPC, ttk, tkFileDialog
from Tkinter import *
import Tkinter as tk
from datetime import datetime
from opcua import ua, uamethod, Server

No handlers could be found for logger "opcua.client.client"


In [3]:
opc = OpenOPC.client() 
opc.servers()
opc_list=opc.servers()

In [4]:
window = tk.Tk()
window.title('OPC Converter')
window.geometry('650x512')

''

In [5]:
#下拉菜单 选择OPC_DA服务器
L1 = tk.Label(window,text='Select a OPC_DA server:', width=30).grid(row = 0, column = 0)
C1 = ttk.Combobox(window,value=(opc.servers()), width = 40).grid(row = 0, column = 1)

In [6]:
#配置OPC_UA服务器
def SelectCaPath():
    path_Ca = tkFileDialog.askopenfilename(filetypes=[("der","der")])
    pathCa.set(path_Ca)
def SelectPkPath():
    path_Pk = tkFileDialog.askopenfilename(filetypes=[("pem","pem")])
    pathPk.set(path_Pk)
pathCa = StringVar()
pathPk = StringVar()
L2 = tk.Label(window, text = 'Set OPC_UA Certificate:', width = 30).grid(row = 1, column = 0)
E1 = tk.Entry(window, textvariable = pathCa, width = 40).grid(row = 1, column = 1)
B1 = tk.Button(window, text = 'Select file', command = SelectCaPath).grid(row = 1, column = 2)
L3 = tk.Label(window, text = 'Set OPC_UA Private_Key:', width = 30).grid(row = 2, column = 0)
E2 = tk.Entry(window, textvariable = pathPk, width = 40).grid(row = 2, column = 1)
B2 = tk.Button(window, text = 'Select file', command = SelectPkPath).grid(row = 2, column = 2)
L4 = tk.Label(window, text = 'Set OPC_UA URI:', width = 30).grid(row = 3, column = 0)
E3 = tk.Entry(window, width = 40).grid(row = 3, column = 1)
L5 = tk.Label(window, text = 'Set OPC_UA End_Point:', width = 30).grid(row = 4, column = 0)
E4 = tk.Entry(window, width = 40).grid(row = 4, column = 1)

In [7]:
#启动转发
B3 = tk.Button(window, text = 'Start').grid(row=5,column=1)

In [8]:
 #开启窗口
window.mainloop()